# About

A sketch of the [machine translation tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) from the [PyTorch Tutorials](https://pytorch.org/tutorials/index.html).

It's the closest analogy to automated document conversion, so this is an attempt to study its internals.

## Model

This is a sequence-to-sequence (seq2seq) model, consisting of a pair of [recurrent neural networks (RNN)](https://en.wikipedia.org/wiki/Recurrent_neural_network), the encoder and decoder. 

The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

![diagram](images/seq2seq.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Encoder

This is the first half of the RNN seq2seq network, aka [Encoder Decoder network](https://arxiv.org/pdf/1406.1078v3.pdf).

For every input (word token, in this example) the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.

![encoder](images/encoder-network.png)

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        # using a multi-layer gated recurrent unit (GRU) 
        # as an improvement over long short-term memory (LSTM) 
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input_layer, hidden_layer):
        embedded = self.embedding(input_layer).view(1, 1, -1)
        output_layer, hidden_layer = self.gru(embedded, hidden_layer)
        return output_layer, hidden_layer
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Decoder

The is the second half of the model, another RNN that takes the encoder's output vectors as its input, and outputs a sequence of words to create the translation:

![decoder](images/decoder-network.png)

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_layer, hidden_layer):
        output_layer = F.relu(self.embedding(input).view(1, 1, -1))
        output_layer, hidden_layer = self.gru(output_layer, hidden_layer)
        output_layer = self.softmax(self.out(output_layer[0]))
        return output_layer, hidden_layer
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention Decoder

Without this, the context vector which is passed betweeen the encoder and decoder carries the burden of encoding the entire sentence.

The attention RNN allows the decoder network to "focus" on a different part of the encoder's outputs for every step of the decoder's own outputs.

![diagram](images/attention-decoder.png)

Calculating the attention weights is done with another feed-forward layer, `attn`, using the decoder's input and hidden state as inputs.

Rather than using [bucketing](https://www.kaggle.com/bminixhofer/speed-up-your-rnn-with-sequence-bucketing) for handling inputs of variable length in the training data, this example chooses a maximum sentence length that gets applied to all inputs: sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few.

![diagram](images/attention-decoder-network.png)

In [5]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout, max_length):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length
        sef.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attention = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attention_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input_layer, hidden_layer, encoder_output):
        embedded = self.dropout(self.embedding(input_layer).view(1, 1, -1))
        attention_weights = F.softmax(self.attention(torch.cat((embedded[0], hidden_layer[0]), 1)), dim=1)
        attention_applied = torch.bmm(attention_weights.unsqueeze(0), encoder_output.unsqueeze(0))
        output_layer = self.attention_combine(torch.cat((embedded[0], attention_applied[0]), 1)).unsqueeze(0)
        output_layer, hidden_layer = self.gru(F.relu(output_layer), hidden_layer)
        output_layer = F.log_softmax(self.out(output_layer[0]), dim=1)
        return output_layer, hidden_layer, attention_weights
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Putting it all together

Using the tab-delimited examples from [Anki](https://www.manythings.org/anki/) to see if it's possible to produce a simple English to Japanese translation service.

### Sidebar: parsing Japanese text in python

This proved to be surprisingly complicated. 

[KyTea](http://www.phontron.com/kytea/) would not install until going down to version `0.3.2`, and even then would not run properly.

[This guide](images/japanese-spacy-and-mecab.pdf) (archived from the [original](https://www.dampfkraft.com/nlp/japanese-spacy-and-mecab.html)) was helpful, but I needed a few more tweaks to make it work:

* `sudo ldconfig` gets around the `error while loading shared libraries: libmecab.so.2` building `unidic-mecab-2.1.2_src` (hat tip to [tatsuyaoiw](http://tatsuyaoiw.hatenablog.com/entry/20120414/1334397985))
* edited `/usr/local/etc/mecabrc` so that `dicdir` is defined as `dicdir = /usr/local/lib/mecab/dic/unidic`
* after `pip install mecab-python3==0.7` and `pip install spacy` I also needed `pip install fugashi`

In [6]:
!wget -P /tmp https://www.manythings.org/anki/jpn-eng.zip

--2020-05-12 19:27:55--  https://www.manythings.org/anki/jpn-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:3037::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2127866 (2.0M) [application/zip]
Saving to: ‘/tmp/jpn-eng.zip’

jpn-eng.zip         100%[===================>]   2.03M   633KB/s    in 3.3s    

2020-05-12 19:27:58 (633 KB/s) - ‘/tmp/jpn-eng.zip’ saved [2127866/2127866]



In [7]:
!unzip -d /tmp /tmp/jpn-eng.zip

Archive:  /tmp/jpn-eng.zip
  inflating: /tmp/jpn.txt            
  inflating: /tmp/_about.txt         


In [8]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name, parser):
        self.name = name
        self.parser = parser
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
        
    def add_sentence(self, sentence):
        for word in self.parser(sentence):
            self.add_word(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [9]:
"""
Define the Japanese language parser
"""

import spacy
ja = spacy.blank('ja')

def parse_japanese(sentence):
    return [word for word in ja(sentence)]

"""
English can be more complicated than this, 
but this suffices for this simple model
"""

def parse_english(sentence):
    return sentence.split(' ')

In [10]:
def parse_language_pairs(input_file='/tmp/jpn.txt'):
    pairs = [] # list of (eng, jpn) pairs
    for line in open(input_file, encoding='utf-8').read().splitlines():
        parts = line.split('\t')
        if len(parts) > 1:
            pairs.append((parts[0].strip(), parts[1].strip()))
    return pairs

In [11]:
samples = parse_language_pairs()

In [12]:
# define the training, validation, testing data sets as 50 : 25 : 25
sample_size = len(samples)
training = samples[0:sample_size // 2]
validation = samples[sample_size // 2:(sample_size // 2)+(sample_size // 4)]
testing = samples[(sample_size // 2)+(sample_size // 4):]
print("{} samples, {} training, {} validation, {} testing"
      .format(sample_size, len(training), len(validation), len(testing)))

48955 samples, 24477 training, 12238 validation, 12240 testing
